











# ORB Detector Parameter Tuning 
## Setup the required functions and input images

In [ ]:
import numpy as np
import cv2
import sys, os
from matplotlib import pyplot as plt
sys.path.insert(0, os.path.join(globals()['_dh'][0], os.pardir,'helper_functions'))
from vslam_helper import *

%matplotlib inline

K = np.array([[699.33112889, 0.0, 403.23876197],
              [0.0, 693.66457792, 293.40739086],
              [0.0, 0.0, 1.0]])

D =  np.array([-2.78089511e-01,  1.30037134e-01, -1.17555797e-04, -1.55337290e-04, -4.34486330e-02])

data_path = os.path.join(os.pardir,'data')

img1 = cv2.imread(os.path.join(data_path, 'cervino_800x600/G0057821.jpg'),cv2.IMREAD_COLOR)
img2 = cv2.imread(os.path.join(data_path, 'cervino_800x600/G0057826.jpg'),cv2.IMREAD_COLOR)

gr1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gr2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

clahe = cv2.createCLAHE(clipLimit=40.0, tileGridSize=(4,3))
gr1 = clahe.apply(gr1)
gr2 = clahe.apply(gr2)

# Read image masks to mask iceberg sections
m1 = cv2.imread(os.path.join(data_path, 'cervino_800x600/G0057821_mask.png'),cv2.IMREAD_GRAYSCALE)
m2 = cv2.imread(os.path.join(data_path, 'cervino_800x600/G0057826_mask.png'),cv2.IMREAD_GRAYSCALE)

## Feature Detection

In [ ]:
TILE_KP = True
RADIAL_NON_MAX = True
RADIAL_NON_MAX_RADIUS = 5
TILEY = 4
TILEX = 3

detector = cv2.ORB_create(nfeatures=10000, scaleFactor=2.0, nlevels=4, edgeThreshold=31, 
                           firstLevel=0, WTA_K=2, scoreType=cv2.ORB_HARRIS_SCORE,  
                           patchSize=31, fastThreshold=15)
'''
detector = cv2.xfeatures2d.SIFT_create(nfeatures = 10000, nOctaveLayers = 3, contrastThreshold = 0.01, 
                                       edgeThreshold = 20, sigma = 1.6)
'''
# find the keypoints and descriptors with ORB
kp1 = detector.detect(gr1,m1)
print("Detected kps: ",len(kp1))
kp2 = detector.detect(gr2,m2)

if TILE_KP:
    kp1 = tiled_features(kp1, gr1.shape, TILEY, TILEX)
    kp2 = tiled_features(kp2, gr2.shape, TILEY, TILEX)
    print ("Points after tiling supression: ",len(kp1))

if RADIAL_NON_MAX:
    kp1 = radial_non_max_kd(kp1,RADIAL_NON_MAX_RADIUS)
    kp2 = radial_non_max_kd(kp2,RADIAL_NON_MAX_RADIUS)
    print ("Points after radial supression: ",len(kp1))

img_out1 = draw_keypoints(cv2.cvtColor(gr1, cv2.COLOR_GRAY2BGR),kp1, color = (255, 255, 0))
img_out2 = draw_keypoints(cv2.cvtColor(gr2, cv2.COLOR_GRAY2BGR),kp2, color = (255, 255, 0))


fig, [ax1, ax2] = plt.subplots(1,2,dpi=200)
fig.suptitle('800x600 Yellow circle defult setting patchSize=31')
ax1.axis("off")
ax1.imshow(img_out1)
ax2.axis("off")
ax2.imshow(img_out2)
fig.subplots_adjust(0,0,1,1,0.0,0.0)
plt.show()

## Feature matching

In [ ]:
kp1, des1 = detector.compute(gr1,kp1)
kp2, des2 = detector.compute(gr2,kp2)
'''
FLANN_INDEX_LSH = 6
FLANN_INDEX_KDTREE = 1
matcher = cv2.FlannBasedMatcher(dict(algorithm = FLANN_INDEX_LSH, table_number = 6, key_size = 20,
                                   multi_probe_level = 2), dict(checks=100))
'''
matcher = cv2.BFMatcher(cv2.NORM_HAMMING2, crossCheck=True)
#matches12_knn = matcher.knnMatch(des1,des2,k=2)
matches12 = matcher.match(des1,des2)
#matches12 = [item for sublist in matches12_knn for item in sublist]
print("Found matches: ",len(matches12))

kp1_match_12 = np.array([kp1[mat.queryIdx].pt for mat in matches12])
kp2_match_12 = np.array([kp2[mat.trainIdx].pt for mat in matches12])

#matches12 = sorted(matches12, key = lambda x:x.distance)
#matches12 = matches12[:(int)(len(matches12)*.75)]

kp1_match_12_ud = cv2.undistortPoints(np.expand_dims(kp1_match_12,axis=1),K,D)
kp2_match_12_ud = cv2.undistortPoints(np.expand_dims(kp2_match_12,axis=1),K,D)

E_12, mask_e_12 = cv2.findEssentialMat(kp1_match_12_ud, kp2_match_12_ud, focal=1.0, pp=(0., 0.), 
                                       method=cv2.RANSAC, prob=0.9999, threshold=0.001)

print("After essential: ", np.sum(mask_e_12))

img_valid = draw_feature_tracks(gr1,kp1,gr2,kp2, matches12, mask_e_12, display_invalid=True, color=(0, 255, 0))
#img_valid = displayMatches(gr1,kp1,gr2,kp2, matches12, mask_e_12, display_invalid=False, color=(0, 255, 0))

fig, ax= plt.subplots(dpi=200)
plt.title('ORB detector with BF Matcher and Cheirality filter')
plt.axis("off")
plt.imshow(img_valid)
plt.show()
